In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class GATLayer(nn.Module):
    def __init__(self):
        super(GATLayer, self).__init__()
    def forward(self, input, adj):
        print("")

In [3]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes, in_features)

# Linear transformation
h = torch.mm(input, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [6]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)
leakyRelu = nn.LeakyReLU(0.2)
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)],
                   dim=1).view(N, -1, 2 * out_features)

torch.Size([4, 1])


In [16]:
e = leakyRelu(torch.matmul(a_input, a).squeeze(2))

In [18]:
adj = torch.randint(2, (3,3))
zero_vec = -9e15*torch.ones_like(e)
zero_vec.shape

torch.Size([3, 3])

In [19]:
attention = torch.where(adj > 0, e, zero_vec)
adj

tensor([[1, 1, 0],
        [1, 0, 0],
        [1, 1, 1]])

In [20]:
e

tensor([[1.2343, 1.2071, 0.3332],
        [1.8328, 1.8056, 0.9317],
        [1.4469, 1.4197, 0.5458]], grad_fn=<LeakyReluBackward0>)

In [21]:
zero_vec

tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])

In [22]:
attention

tensor([[ 1.2343e+00,  1.2071e+00, -9.0000e+15],
        [ 1.8328e+00, -9.0000e+15, -9.0000e+15],
        [ 1.4469e+00,  1.4197e+00,  5.4579e-01]], grad_fn=<SWhereBackward>)

In [23]:
attention = F.softmax(attention, dim=1)

In [24]:
attention

tensor([[0.5068, 0.4932, 0.0000],
        [1.0000, 0.0000, 0.0000],
        [0.4203, 0.4090, 0.1707]], grad_fn=<SoftmaxBackward>)

In [25]:
h_prime = torch.matmul(attention, h)

In [27]:
print(h_prime,"\n", h)

tensor([[0.5032, 0.8943],
        [0.0562, 1.1620],
        [0.4706, 0.7691]], grad_fn=<MmBackward>) 
 tensor([[0.0562, 1.1620],
        [0.9626, 0.6192],
        [0.3122, 0.1611]], grad_fn=<MmBackward>)


In [29]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.concat = concat
        
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        self.leakyRelu = nn.LeakyReLU(self.alpha)
        pass
    
    def forward(self, input, adj):
        h = torch.mm(input, self.W)
        N = h.size()[0]
        
        a_input = torch.car([h.repeat(1,N).view(N*N, -1), 
                            h.repeat(N, 1)], dim=1).view(N, -1, 2*self.out_features)
        e = self.leakyRelu(torch.matmul(a_input, a))
        
        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj >0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        
        h_prime = torch.matmul(attention, h)
        
        if self.concat :
            return F.elu(h_prime)
        else:
            return h_prime

In [33]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import matplotlib.pyplot as plt

name_data = 'Cora'
dataset = Planetoid(root='data/'+name_data, name=name_data)
dataset.transform = T.NormalizeFeatures()

In [34]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head,
                            dropout=0.6)
        self.conv2=GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                          heads=self.out_head, dropout=0.6)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x=F.dropout(x, p=0.6, training=self.training)
        x=self.conv1(x, edge_index)
        x=F.elu(x)
        x=F.dropout(x,p=0.6, training=self.training)
        x=self.conv2(x,edge_index)
        
        return F.log_softmax(x, dim=1)
        

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT().to(device)
data = dataset[0].to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, 
                            weight_decay=5e-4)

model.train()
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    if epoch % 200 == 0:
        print(loss)
    
    loss.backward()
    optimizer.step()

tensor(1.9458, grad_fn=<NllLossBackward>)
tensor(0.7380, grad_fn=<NllLossBackward>)
tensor(0.6137, grad_fn=<NllLossBackward>)
tensor(0.5559, grad_fn=<NllLossBackward>)
tensor(0.4587, grad_fn=<NllLossBackward>)


In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
